# Lag

##### Copyright 2022, Group 4


# Importing the libraries:


In [ ]:
install.packages('zoo');install.packages('PlayerRatings')
library(dplyr);library(zoo);library(PlayerRatings);library(tidyr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




#Load data:

In [ ]:
getwd()
list.files()

[1] "/content"

[1] "sample_data"

In [ ]:
advancedboxscores <- read.csv('/content/out.csv')
playerclusters <- read.csv('/content/players.csv')

#Preprocess games data


###Create home court advantage, create lag columns:

In [ ]:
preprocess_games <- advancedboxscores %>%
                    mutate(HOME_ADVANTAGE = ifelse(grepl('vs.', MATCHUP),1, 0)) %>%
                    arrange(TEAM_ABBREVIATION, GAME_DATE) %>% 
                    group_by(TEAM_ABBREVIATION, SEASON_YEAR) %>% 
                    mutate(WL = as.factor(WL),
                          SEASON_YEAR = as.numeric(sub("\\-.*", "", SEASON_YEAR))+1,
                          WL_LAG = rollsumr(as.numeric(WL) -1, k = 10, fill = NA),
                          EFG_PCT_LAG = rollmeanr(EFG_PCT, k = 10, fill = NA),
                          FTA_RATE_LAG = rollmeanr(FTA_RATE, k = 10, fill = NA),
                          TM_TOV_PCT_LAG = rollmeanr(TM_TOV_PCT, k = 10, fill = NA),
                          OREB_PCT_LAG = rollmeanr(OREB_PCT, k = 10, fill = NA),
                          OPP_EFG_PCT_LAG = rollmeanr(OPP_EFG_PCT, k = 10, fill = NA),
                          OPP_FTA_RATE_LAG = rollmeanr(OPP_FTA_RATE, k = 10, fill = NA),
                          OPP_TOV_PCT_LAG = rollmeanr(OPP_TOV_PCT, k = 10, fill = NA),
                          OPP_OREB_PCT_LAG = rollmeanr(OPP_OREB_PCT, k = 10, fill = NA)) %>%
                    ungroup()


###Create and merge glicko rating:

In [ ]:
g2r <- glicko2(data.frame(advancedboxscores$X, 
                          advancedboxscores$TEAM_ABBREVIATION,
                          sub('.*@ |.*vs. ', '', advancedboxscores$MATCHUP),
                          ifelse(advancedboxscores$WL == 1,0,1)),
              history = TRUE)
g2rtrim <- data.frame(g2r[["history"]][1,,1])
g2rtrim$gamenumber <- seq.int(nrow(g2rtrim))
colnames(g2rtrim) <- c("rating", "gamenumber")
preprocess_games <- merge(preprocess_games, g2rtrim, by.x = "X", by.y ="gamenumber")

###Remove unneeded columns:

In [ ]:
preprocess_games <- preprocess_games %>% select(-(1 | contains('rank')))

###Create games dataframe:

In [ ]:
games <- preprocess_games

#Preprocess players:


###Pivot clusters, rename columns, and sum number of players per cluster:

In [ ]:
player_stats <- playerclusters %>%
        group_by(Year, Tm) %>%
        mutate(mean_VORP = mean(VORP),
               mean_BPM = mean(BPM)) %>%
        ungroup() %>%
        distinct(Year, Tm, mean_VORP, mean_BPM)

player_clusters <- playerclusters %>% 
                      mutate(value = 1) %>% 
                      spread(cluster, value,  fill = 0 ) %>% 
                      rename(Cluster1 = `1`, Cluster2 = `2`, Cluster3 = `3`) %>%
                      select (Year, Tm, Cluster1, Cluster2, Cluster3) %>%
                      group_by(Year, Tm) %>% 
                      summarise(across(everything(), ~ sum(., is.na(.), 0))) %>%
                      ungroup()
preprocess_players <- merge(player_stats, player_clusters, by = c('Year','Tm'))

`summarise()` has grouped output by 'Year'. You can override using the
`.groups` argument.


###Some team abbreviations need to be updated:

In [ ]:
preprocess_players <- preprocess_players %>%
                      mutate(Tm = case_when(Tm == 'PHO' ~ 'PHX',
                                            Tm == 'BRK' ~ 'BKN',
                                            Tm == 'CHO' ~ 'CHA',
                                            TRUE ~ Tm))

players <- preprocess_players
output <- merge(games, players, by.x = c('SEASON_YEAR', 'TEAM_ABBREVIATION'), 
                by.y = c('Year','Tm'))#, all = TRUE)

###Save Output file as csv:

In [ ]:
write.csv(output, '/content/preprocess-games.csv')
#rm(list=ls(all=TRUE))